In [1]:
!pip install keras_facenet

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install joblib

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install mtcnn

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install django

Defaulting to user installation because normal site-packages is not writeable


In [11]:
def face_extract(filename, required_size=(160, 160)):
    image = Image.open(filename)
    image = image.convert('RGB')
    pixels = np.asarray(image)

    faces = detector.detect_faces(pixels)
#     x, y, width, height = faces[0]['box']
#     x1, y1 = abs(x) + width, abs(y) + height
#     face = pixels[y:y1, x:x1]

#     image = Image.fromarray(face)  # use new variable(i.e image/or any other)
#     image = image.resize(required_size)
#     face_array = np.asarray(image)
    return faces
fe=face_extract('data/val/Deependra/myimage1.png')
fe


1/1 [==============================] - 0s 23ms/step


[{'box': [154, 91, 128, 174],
  'confidence': 0.9883503913879395,
  'keypoints': {'left_eye': (193, 154),
   'right_eye': (253, 157),
   'nose': (223, 194),
   'mouth_left': (198, 228),
   'mouth_right': (243, 229)}}]

In [6]:
# packages
import os
import joblib
import keras_facenet
import cv2
import mtcnn
import pandas as pd
from keras_facenet import FaceNet
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from mtcnn.mtcnn import MTCNN
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import django
from django.db import models
print('hi')
img = cv2.imread('lataji_1.jpg')
# plt.imshow(img,cmap='gray',interpolation='bicubic')
# plt.show()

model_svc = SVC(kernel='linear', probability=True)


class Keras_Facenet():
    
    def face_extract(self, filename, required_size=(160, 160)):
        image = Image.open(filename)
        image = image.convert('RGB')
        pixels = np.asarray(image)
        detector = MTCNN()
        faces = detector.detect_faces(pixels)
        x, y, width, height = faces[0]['box']
        x1, y1 = abs(x) + width, abs(y) + height
        face = pixels[y:y1, x:x1]

        image = Image.fromarray(face)  # use new variable(i.e image/or any other)
        image = image.resize(required_size)
        face_array = np.asarray(image)
        return face_array

    def load_faces(self, dir):
        faces = list()
        for subdir in os.listdir(dir):
            path = dir + subdir
            face = self.face_extract(path)
            faces.append(face)
        return faces

    def create_dataset(self, dir):  # images
        X = list()
        Y = list()
        for subdir in os.listdir(dir):  # subdir==harsh
            path = dir + subdir + '/'
            faces = self.load_faces(path)
            labels = [subdir for i in range(len(faces))]
            # print("loaded %d sample for class: %s" % (len(faces), subdir))  # print progress
            X.extend(faces)
            Y.extend(labels)
        # return np.asarray(X),np.asarray(Y)
        return X, Y

    def __call__(self, train_path, test_path, filename):

        facenet = Keras_Facenet()
        trainx, trainy = facenet.create_dataset(train_path)  # list
        testx, testy = facenet.create_dataset(test_path)
        facenet_model = FaceNet()
        train_emb = facenet_model.embeddings(trainx)
        print(train_emb)
        print(train_emb.shape)

        test_emb = facenet_model.embeddings(testx)
        x_train, y_train = np.asarray(train_emb), np.asarray(trainy)
        x_test, y_test = np.array(test_emb), np.asarray(testy)
        from numpy import save
        save('y_train.npy',y_train)
        print("Dataset: train=%d, test=%d" % (train_emb.shape[0], test_emb.shape[0]))
        # normalize input vectors
        in_encoder = Normalizer()
        emdTrainX_norm = in_encoder.transform(train_emb)
        emdTestX_norm = in_encoder.transform(test_emb)
        # label encode targets
        out_encoder = LabelEncoder()
        out_encoder.fit(y_train)
        trainy_enc = out_encoder.transform(y_train)
        testy_enc = out_encoder.transform(y_test)
        # fit model
        joblib.dump(out_encoder,'out_encoder.sav')
        save('testy_enc.npy',testy_enc)
        model_svc.fit(emdTrainX_norm, trainy_enc)
        # predict
        yhat_train = model_svc.predict(emdTrainX_norm)
        yhat_test = model_svc.predict(emdTestX_norm)
        # score
        score_train = accuracy_score(trainy_enc, yhat_train)
        score_test = accuracy_score(testy_enc, yhat_test)
        # summarize
        print('Accuracy: train=%.3f, test=%.3f' % (score_train * 100, score_test * 100))
        filename_path = 'finalized_model.sav'
        joblib.dump(model_svc, filename_path)
        print('model ran successfully')

        #         def show_image(self,facenet_model,filename):#filename='data/val/Deependra/WhatsApp Image 2022-10-03 at 5.42.53 PM (2).jpeg'
        face = self.face_extract(filename)
        facenew = np.expand_dims(face, axis=0)
        face_emb = facenet_model.embeddings(list(facenew))
        print('emb_shape', face_emb.shape)
        # face_emb=np.expand_dims(face_emb,axis=0)
        yhat_face = model_svc.predict(face_emb)
        yhat_prob = model_svc.predict_proba(face_emb)
       
        name=out_encoder.inverse_transform([yhat_face[0]])
        print('class is:',name )
        plt.imshow(face)
        plt.show()

#     person_name=models.CharField(max_length=50)
#     img=models.ImageField()
#     probability=models.FloatField()
#     model_name=models.CharField(max_length=40)

hi


In [7]:
facenet=Keras_Facenet()
facenet('data/train/','data/val/','data/val/Deependra/myimage2.png')

1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 16ms/step


2/2 [==============================] - 0s 2ms/step


3/3 [==============================] - 0s 5ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 14ms/step


IndexError: list index out of range

In [26]:
from numpy import load
data1=load('y_train.npy')
data1[0]

'ben_afflek'

In [27]:
from numpy import load
data=load('testy_enc.npy')
data[0]

1

In [ ]:
joblib.dump(model_svc,'keras_facenet_model.pkl')

In [10]:
model=joblib.load('finalized_model.sav')

In [13]:
filename='data/val/Deependra/WhatsApp Image 2022-10-03 at 5.30.22 PM.jpeg'
facenet=FaceNet()
model=Keras_Facenet()
face = model.face_extract(filename)
facenew = np.expand_dims(face, axis=0)
face_emb = facenet.embeddings(list(facenew))


1/1 [==============================] - 5s 5s/step


In [14]:
face_emb

array([[ 0.03281442, -0.00815369,  0.02555412,  0.06263109,  0.00402384,
        -0.01287944,  0.04804228,  0.01557051,  0.06485348,  0.01765059,
         0.04651985,  0.0222063 ,  0.04857896, -0.02932801, -0.05322829,
         0.03147658, -0.01485197,  0.05437467, -0.0201363 ,  0.02433932,
         0.00831702, -0.02174938,  0.02988574, -0.04219646, -0.06151272,
         0.04016986, -0.02831053, -0.04418784, -0.02541473, -0.05087055,
         0.09649095,  0.06747718, -0.00304049, -0.09542526,  0.04619103,
        -0.03760095,  0.03713369, -0.05759285, -0.05343715,  0.01457255,
         0.02528671, -0.02014452, -0.00474314,  0.00140079, -0.02556697,
        -0.02606297, -0.04956662,  0.07695492, -0.04025707, -0.03328012,
         0.003967  , -0.02383374, -0.02487873,  0.00128212, -0.02433814,
        -0.00697188,  0.02803588, -0.03570313, -0.03584859,  0.02773031,
        -0.00252242,  0.05740833, -0.038757  ,  0.04037654, -0.07315186,
        -0.01061345, -0.02915164, -0.07978453,  0.0

In [15]:
yhat_face = model_svc.predict(face_emb)
yhat_prob = model_svc.predict_proba(face_emb)

In [18]:
yhat_face[0]

0

In [17]:
yhat_prob

array([[0.88996834, 0.01355501, 0.02484981, 0.00796617, 0.03584626,
        0.0278144 ]])

In [18]:
type(yhat_prob)

numpy.ndarray

In [22]:
yhat_prob.shape

(1, 6)

In [21]:
np.argmax(yhat_prob,axis=1)

array([0], dtype=int64)

In [30]:
!pip install pyautogui

Defaulting to user installation because normal site-packages is not writeable
  Using cached PyAutoGUI-0.9.53.tar.gz (59 kB)
  Using cached PyMsgBox-1.0.9.tar.gz (18 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached pytweening-1.0.4.tar.gz (14 kB)
  Using cached PyScreeze-0.1.28.tar.gz (25 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached PyGetWindow-0.0.9.tar.gz (9.7 kB)
  Using cached MouseInfo-0.1.3.tar.gz (10 kB)
  Using cached Py

In [31]:

# Python program to take
# screenshots
  
  
import numpy as np
import cv2
import pyautogui
   
  
# take screenshot using pyautogui
image = pyautogui.screenshot()
   
# since the pyautogui takes as a 
# PIL(pillow) and in RGB we need to 
# convert it to numpy array and BGR 
# so we can write it to the disk
image = cv2.cvtColor(np.array(image),
                     cv2.COLOR_RGB2BGR)
   
# writing it to the disk using opencv
cv2.imwrite("image1.png", image)

True